In [1]:
import pandas as pd
import re
import numpy as np
import fsspec
import os
import rasterio as rio
import matplotlib.pyplot as plt
import boto3

In [2]:
def rastermath(ensemble_list, iteration):
    
    # Purpose: To calculate the mean across X ensembles for each ith pixel
    
    # Returns a tuple with the first element the array, and the second the metadata of the the geoTiff file
    
    array_list=[]

    for j in range(0, len(ensemble_list)):
        dataset = ensemble_list[j]
        
        raster_array_read = dataset.read(iteration+1, masked=False) # fixed from masked=False to masked=True on 8/17/2020 - this applied to the gridMET data
        raster_meta = dataset.profile

        array_list.append(raster_array_read)
        
        array_mean = array_list[0]
        
    return array_mean, raster_meta

In [3]:
def grepfxn(pattern,files):
    # Purpose: this function 
    precip_asc_df2=pd.DataFrame(files)
    idx_asc_in2 = []
    for i, precipfile in precip_asc_df2.iterrows():
        val = re.findall(pattern, precipfile.iloc[0])
        if len(val) > 0:
            idx_asc_in2.append(i)
            
    ascfileround2 = [files[i] for i in idx_asc_in2]
    return ascfileround2

In [4]:
def write_geotiff(data,meta,var_name,doy_name,folder):
    
    # Now we need to convert all of these arrays back to rasterio geoTIFFs
    
    os.chdir(folder+'/'+var_name)
    
    doy_name_fill = str(doy_name).zfill(3)
    
    filename = var_name +'_DOY' +str(doy_name_fill) +'.tif'
    with rio.open(filename, 'w', **meta) as dst:
        dst.write(data, 1)
        
    return filename

In [5]:
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)

In [6]:
averaging_years = np.arange(1950, 2110, 10).tolist();print(averaging_years)

# num_years = 10

# print(np.arange(1950, 1960, 1))
# averaging_years[0]
# averaging_years[1]

[1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]


In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
all_files = fs.find('dev-et-data/in/DelawareRiverBasin/ETo/')

In [ ]:
'''This is the meat of the code that exports time averaged reference ET into the cloud data bucket'''

# averaging_years2 = averaging_years[0:2]

block_averaged_list = []
for i in range(0,len(averaging_years)-1): # loop by the defined averaging block e.g. 1950, 1960, 1970 etc. for 10 years
    year_block_annual = np.arange(averaging_years[i], averaging_years[i+1], 1) # create list of annual to loop by next
#     print(year_block_annual)
    block_daily_annual_list = []        
        
    for year_annual_loop in range(0, len(year_block_annual)): # loop by each year in the defined blocking period

        # get only the matching filenames for that year
        year_annual_loop_geotiff = grepfxn(str(year_block_annual[year_annual_loop]),all_files)
            
        for year_daily_loop in range(0, len(year_annual_loop_geotiff)): # loop by all days within the given year

            year_daily_loop_geotiff = year_annual_loop_geotiff[year_daily_loop]

            daily_constructed_filename = "s3://" + year_daily_loop_geotiff 

            open_daily_geotiff = rio.open(daily_constructed_filename)

            block_daily_annual_list.append(open_daily_geotiff)
            
    averaged_array = rastermath(block_daily_annual_list,0)
        
#     block_averaged_list.append(averaged_array)
        
    averaged_array[1]['count']=1
    
    # Much like for the original reference evapotranspiration images, write the geotiffs to the local mini-pangeo
    # and then push that to the cloud (while deleting the copy in mini-pangeo)
    
    os.chdir('/home/jupyter-rouze')
    gTIFF_filename = write_geotiff(data=averaged_array[0],meta=averaged_array[1],var_name='reference_evapotranspiration',
                                   startyear=averaging_years[i],endyear=averaging_years[i+1],folder='in')

    local_file = 'in' +'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
    bucket = 'dev-et-data'

    bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average/' + gTIFF_filename 

    os.chdir('/home/jupyter-rouze')
    s3_push_delete_local(local_file, bucket, bucket_filepath)
    
    block_daily_annual_list = []     

In [ ]:
averaging_years

In [ ]:
os.chdir('/home/jupyter-rouze')
gTIFF_filename = write_geotiff(data=averaged_array[0],meta=averaged_array[1],var_name='reference_evapotranspiration',
                               startyear=averaging_years[i],endyear=averaging_years[i+1],folder='in')

In [ ]:
print(averaged_array)
print(year_block_annual)
print(averaged_array[0])
averaged_array[1]['count']=1

# Much like for the original reference evapotranspiration images, write the geotiffs to the local mini-pangeo
# and then push that to the cloud (while deleting the copy in mini-pangeo)

In [ ]:
os.chdir('/home/jupyter-rouze')
gTIFF_filename = write_geotiff(data=averaged_array,meta=averaged_array[1],var_name='reference_evapotranspiration',
                               doy=doy_loop,year=year_loop,folder=output_folder)

local_file = output_folder+'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
bucket = 'dev-et-data'

bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average/'+ str(year_loop)  + '/' + gTIFF_filename 

os.chdir('/home/jupyter-rouze')
s3_push_delete_local(local_file, bucket, bucket_filepath)

block_daily_annual_list = []
        

In [ ]:
block_averaged_list[0]

In [ ]:
block_averaged_list[1]

In [ ]:
# print(block_averaged_list[0][0])
# averaged_array = rastermath(block_daily_annual_list,0)
# print(averaged_array)
# print(block_averaged_list)
# print(block_averaged_list[0])
plt.imshow(block_averaged_list[0][0])

In [ ]:
rastermath(block_daily_annual_list,0)[0]

In [ ]:
block_daily_annual_list

In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
all_files = fs.find('dev-et-data/in/DelawareRiverBasin/ETo/')
# print(all_files)
# print(grepfxn('1964',all_files))

year_block_annual = np.arange(averaging_years[0], averaging_years[0+1], 1)
print(year_block_annual)
print(str(year_block_annual[5]))

In [ ]:
year_block_annual

In [ ]:
year_block_annual_list = []

year_annual_loop_geotiff = grepfxn(str(year_block_annual[5]),all_files)
print(year_annual_loop_geotiff)
len(year_annual_loop_geotiff)

In [ ]:
year_annual_loop_geotiff = grepfxn(str(year_block_annual[5]),all_files)
print(year_annual_loop_geotiff)

In [ ]:
year_daily_loop_geotiff = year_annual_loop_geotiff[0]
print(year_daily_loop_geotiff)

In [ ]:
daily_constructed_filename = "s3://" + year_daily_loop_geotiff 

print(daily_constructed_filename)

In [ ]:
test=rio.open(daily_constructed_filename)
print(test)

In [ ]:
year_daily_loop_geotiff2 = year_annual_loop_geotiff[1]
daily_constructed_filename2 = "s3://" + year_daily_loop_geotiff2 
test2=rio.open(daily_constructed_filename2)

In [ ]:
rastermath([test,test2],0)

In [ ]:
# print(year_annual_loop_geotiff)
print(year_daily_loop_geotiff)
print(year_block_annual)
print(year_block_annual[5])
year_annual_loop_geotiff = grepfxn(str(year_block_annual[5]),all_files); print(year_annual_loop_geotiff)

In [ ]:
print(year_annual_loop)
print(str(year_annual_loop))
year_annual_loop_geotiff = grepfxn(str(year_annual_loop),all_files)
# print(year_annual_loop_geotiff)

In [ ]:
print(averaging_years)
print(averaging_years2)
year_block_annual

In [ ]:
year_annual_loop_geotiff

In [ ]:
print(averaging_years)
year_block_annual = np.arange(averaging_years[0], averaging_years[0+1], 1)
range(0, len(year_block_annual))

In [ ]:
year_blocks = np.arange(1950, 2110, 10).tolist();print(year_blocks)
# doy_block_annual = np.arange(0, 366, 1) + 1
# doy_block_annual

In [ ]:
# print(i)
# print(doys_loop)
# print(bucket_filepath)
# print(gTIFF_filename)
# print(doys)
# print(doy_geotiff_list)
# print(gTIFF_filename)
print(doy_year_block)
# print(i)
# print(doys_loop)
# print(doy_year_block[year_doy_loop])

In [ ]:
# doys_loop == 365
# res = [ele for ele in doy_year_block if ele != []] 
# print(res)

doy_year_block[year_doy_loop] == []

In [7]:
'''this is the working code as of 9/15/2020 4:10 pm'''

'''This is the meat of the code that exports time averaged reference ET into the cloud data bucket'''

year_blocks = np.arange(1950, 2110, 10).tolist();print(year_blocks)

fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
all_files = fs.find('dev-et-data/in/DelawareRiverBasin/ETo/')

# averaging_years2 = averaging_years[0:2]

block_averaged_list = []
doy_year_block = []
doy_geotiff_list = []
    
for i in range(0,len(year_blocks)-1): # loop by the defined averaging block e.g. 1950, 1960, 1970 etc. for 10 years
#     year_block_annual = np.arange(averaging_years[i], averaging_years[i+1], 1) # create list of annual to loop by next
#     print(year_block_annual)
    doys = np.arange(0, 366, 1) + 1
#     doy_geotiff_list = []        
        
#     for year_annual_loop in range(0, len(year_block_annual)): # loop by each year in the defined blocking period
    for doys_loop in range(0, len(doys)): # loop by each year in the defined blocking period

        # get only the matching filenames for that year
#         year_annual_loop_geotiff = grepfxn(str(doy_block_annual[year_daily_loop]),all_files)
        # Find the geoTIFFs with the doy.tif file strucure in the data bucket
        # This will include all of the years with the doy
        doy_string_allyears = grepfxn(str(doys[doys_loop]).zfill(3)+'.tif',all_files)
    
        year_block_annual = np.arange(year_blocks[i], year_blocks[i+1], 1) # create list of annual to loop by next
        
        # This wil whittle down the full list of years with the doy to only those in the given year block

        for j in range(0, len(year_block_annual)):
            results = grepfxn(str(year_block_annual[j]),doy_string_allyears)
            doy_year_block.append(results)
    
        for year_doy_loop in range(0, len(doy_year_block)): # loop by all days within the given year
            
            if(doys_loop == 365): # for a leap year

                if doy_year_block[year_doy_loop] == []:
                    print('block year')
                else:
                    year_daily_loop_geotiff = doy_year_block[year_doy_loop][0]
                    daily_constructed_filename = "s3://" + year_daily_loop_geotiff 
                    open_daily_geotiff = rio.open(daily_constructed_filename)
                    doy_geotiff_list.append(open_daily_geotiff)
                
            else:
                year_daily_loop_geotiff = doy_year_block[year_doy_loop][0]
                
                daily_constructed_filename = "s3://" + year_daily_loop_geotiff 
                open_daily_geotiff = rio.open(daily_constructed_filename)
                doy_geotiff_list.append(open_daily_geotiff)

            
        averaged_array = rastermath(doy_geotiff_list,0)

    #     block_averaged_list.append(averaged_array)

        averaged_array[1]['count']=1
        
        averaged_array[1]['crs'] = rio.crs.CRS({"init": "epsg:4326"})
        
        # Much like for the original reference evapotranspiration images, write the geotiffs to the local mini-pangeo
        # and then push that to the cloud (while deleting the copy in mini-pangeo)

        os.chdir('/home/jupyter-rouze')
        gTIFF_filename = write_geotiff(data=averaged_array[0],meta=averaged_array[1],var_name='ETo',
                                       doy_name = doys[doys_loop],folder='in')

    #     print(gTIFF_filename)

        local_file = 'in' +'/' + 'ETo' + '/' + gTIFF_filename 
        bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average_byDOY/' + str(averaging_years[i]) + '_' +str(averaging_years[i+1])+'/'  + gTIFF_filename 
        bucket = 'dev-et-data'

        os.chdir('/home/jupyter-rouze')
        s3_push_delete_local(local_file, bucket, bucket_filepath)

        print(bucket_filepath)

        doy_geotiff_list = []
        doy_year_block = []

[1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY001.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY001.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY002.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY002.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY003.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY003.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY004.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY004.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY005.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY005.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY006.tif
in/DelawareRiverBasin/ETo_Moving_Averag

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY053.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY053.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY054.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY054.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY055.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY055.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY056.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY056.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY057.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY057.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY058.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY058.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY106.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY106.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY107.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY107.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY108.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY108.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY109.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY109.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY110.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY110.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY111.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY111.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY159.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY159.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY160.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY160.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY161.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY161.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY162.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY162.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY163.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY163.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY164.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY164.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY212.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY212.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY213.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY213.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY214.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY214.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY215.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY215.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY216.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY216.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY217.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY217.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_

...

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY116.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY116.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY117.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY117.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY118.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY118.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY119.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY119.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY120.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY120.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY121.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY121.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY169.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY169.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY170.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY170.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY171.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY171.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY172.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY172.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY173.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY173.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY174.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY174.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY222.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY222.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY223.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY223.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY224.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY224.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY225.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY225.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY226.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY226.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY227.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY227.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY275.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY275.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY276.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY276.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY277.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY277.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY278.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY278.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY279.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY279.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY280.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY280.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_

dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY328.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY328.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY329.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY329.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY330.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY330.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY331.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY331.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY332.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY332.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY333.tif
in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_2100/ETo_DOY333.tif
dev-et-data in/DelawareRiverBasin/ETo_Moving_Average_byDOY/2090_

In [ ]:
doy_geotiff_list

In [ ]:
averaged_array

In [ ]:
import rasterio
from rasterio.plot import show
# Random ET image
with rasterio.open('s3://dev-et-data/in/DelawareRiverBasin/ETo_Moving_Average_byDOY/1950_1960/ETo_DOY002.tif') as src:
    print(src.profile)
    show(src)
    array=src.read(1)
print(array)

In [ ]:
year_blocks = np.arange(1950, 2110, 10).tolist();print(averaging_years)

block_averaged_list = []


In [ ]:
# for i in range(0,len(year_blocks)-1): # loop by the defined averaging block e.g. 1950, 1960, 1970 etc. for 10 years

doys = np.arange(0, 366, 1) + 1
doy_geotiff_list = []  

doys


In [ ]:
#     for year_annual_loop in range(0, len(year_block_annual)): # loop by each year in the defined blocking period

i = 0
doys_loop = 1

# for doys_loop in range(1, len(doys)): # loop by each year in the defined blocking period

# get only the matching filenames for that year
#         year_annual_loop_geotiff = grepfxn(str(doy_block_annual[year_daily_loop]),all_files)
# Find the geoTIFFs with the doy.tif file strucure in the data bucket
# This will include all of the years with the doy
doy_string_allyears = grepfxn(str(doys[doys_loop]).zfill(3)+'.tif',all_files)

year_block_annual = np.arange(year_blocks[i], year_blocks[i+1], 1) # create list of annual to loop by next

# This wil whittle down the full list of years with the doy to only those in the given year block
doy_year_block = []
for i in range(0, len(year_block_annual)):
    results = grepfxn(str(year_block_annual[i]),doy_string_allyears)
    doy_year_block.append(results)

for year_doy_loop in range(0, len(doy_year_block)): # loop by all days within the given year

    year_daily_loop_geotiff = doy_year_block[year_doy_loop][0]

    daily_constructed_filename = "s3://" + year_daily_loop_geotiff 

    open_daily_geotiff = rio.open(daily_constructed_filename)

    doy_geotiff_list.append(open_daily_geotiff)

In [ ]:
doy_geotiff_list

In [ ]:
averaged_array = rastermath(block_daily_annual_list,0)

#     block_averaged_list.append(averaged_array)

averaged_array[1]['count']=1

# Much like for the original reference evapotranspiration images, write the geotiffs to the local mini-pangeo
# and then push that to the cloud (while deleting the copy in mini-pangeo)

os.chdir('/home/jupyter-rouze')
gTIFF_filename = write_geotiff(data=averaged_array[0],meta=averaged_array[1],var_name='reference_evapotranspiration',
                               doy_name = doys[doys_loop],folder='in')

print(gTIFF_filename)

In [ ]:
local_file = 'in' +'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average/' + str(averaging_years[0]) + '_' +str(averaging_years[0+1])+'/'  + gTIFF_filename 

print(local_file)
print(bucket_filepath)

In [ ]:
year_blocks

In [ ]:
local_file = 'in' +'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average_byDOY/' + str(year_blocks[i]) + '_' +str(year_blocks[i+1])+'/'  + gTIFF_filename 
bucket = 'dev-et-data'

os.chdir('/home/jupyter-rouze')
s3_push_delete_local(local_file, bucket, bucket_filepath)

block_daily_annual_list = []        

In [ ]:
# doy_block_annual[0].dtype
# n = 4
# n.zfill(3)
# averaging_years = np.arange(1950, 2110, 10).tolist();print(averaging_years)
# averaging_years
doy_annual_loop_geotiff = grepfxn(str(doy_block_annual[0]).zfill(3)+'.tif',all_files)
# len(averaging_years)

year_block_annual = np.arange(averaging_years[0], averaging_years[1], 1) # create list of annual to loop by next
year_block_annual

list_append = []
for i in range(0, len(year_block_annual)):
    results = grepfxn(str(year_block_annual[i]),doy_annual_loop_geotiff)
    list_append.append(results)


In [ ]:
list_append

In [ ]:
year_daily_loop_geotiff
daily_constructed_filename

In [ ]:
block_daily_annual_list=[]
for year_doy_loop in range(0, len(list_append)): # loop by all days within the given year

    year_daily_loop_geotiff = list_append[year_doy_loop][0]

    daily_constructed_filename = "s3://" + year_daily_loop_geotiff 

    open_daily_geotiff = rio.open(daily_constructed_filename)

    block_daily_annual_list.append(open_daily_geotiff)

In [ ]:
block_daily_annual_list

In [ ]:
averaged_array = rastermath(block_daily_annual_list,0)

#     block_averaged_list.append(averaged_array)

averaged_array[1]['count']=1

# Much like for the original reference evapotranspiration images, write the geotiffs to the local mini-pangeo
# and then push that to the cloud (while deleting the copy in mini-pangeo)

os.chdir('/home/jupyter-rouze')
gTIFF_filename = write_geotiff(data=averaged_array[0],meta=averaged_array[1],var_name='reference_evapotranspiration',
                               startyear=averaging_years[i],endyear=averaging_years[i+1],folder='in')

local_file = 'in' +'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
bucket = 'dev-et-data'
bucket_filepath = 'in/DelawareRiverBasin/ETo_Moving_Average_byDOY/' + str(averaging_years[0]) + '_' +str(averaging_years[0+1])+'/'  + 'gTIFF_filename' + '/' + 'DOY_' +str(doy_block_annual[0]) 

print(bucket_filepath)

In [ ]:

os.chdir('/home/jupyter-rouze')
s3_push_delete_local(local_file, bucket, bucket_filepath)

block_daily_annual_list = [] 